In [1]:
import pandas as pd
from selenium import webdriver

In [2]:
driver = webdriver.Chrome('chromedriver')

In [184]:
# 사업보고서 접근경로
감사보고서_경로 = [
    ['2018', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190401002294&dcmNo=6604769'],
    ['2017', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180330003863&dcmNo=6042097'],
    ['2016', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331002110&dcmNo=5529376'],
    ['2015', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330002537&dcmNo=5021094'],
    ['2014', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20150320000348&dcmNo=4522222'],
    ['2013', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20140331002583&dcmNo=4115526'],
]

감사보고서_경로 = pd.DataFrame(감사보고서_경로, columns=['기준년도', 'URL'])

In [207]:
재무상태표_집계 = []
손익계산서_집계 = []
현금흐름표_집계 = []

# 감사보고서 접근
for k in range(감사보고서_경로.shape[0]):
    # 설정
    기준년도 = 감사보고서_경로['기준년도'][k]
    url = 감사보고서_경로['URL'][k]
    driver.get(url)

    # 재무제표 접근
    driver.find_element_by_xpath("//span[contains(text(), '재 무 제 표')]").click()
    url2 = driver.find_element_by_id('ifrm').get_attribute('src')
    driver.get(url2)

    
    # 재무상태표 접근
    text = '재 무 상 태 표'
    재무상태표 = driver.find_element_by_xpath("//span[contains(text(), '{}')]/parent::p/following-sibling::table/following-sibling::table".format(text))
    재무상태표_데이터 = pd.read_html(재무상태표.get_attribute('outerHTML'))[0]

    # 손익계산서 접근
    text = '포 괄 손 익 계 산 서'
    손익계산서 = driver.find_element_by_xpath("//span[contains(text(), '{}')]/parent::p/following-sibling::table/following-sibling::table".format(text))
    손익계산서_데이터 = pd.read_html(손익계산서.get_attribute('outerHTML'))[0]

    # 자본변동표 접근
    #text = '자 본 변 동 표'
    #자본변동표 = driver.find_element_by_xpath("//span[contains(text(), '{}')]/parent::p/following-sibling::table/following-sibling::table".format(text))
    #자본변동표_데이터 = pd.read_html(자본변동표.get_attribute('outerHTML'))[0]

    # 현금흐름표 접근
    text = '현 금 흐 름 표'
    현금흐름표 = driver.find_element_by_xpath("//span[contains(text(), '{}')]/parent::p/following-sibling::table/following-sibling::table".format(text))
    현금흐름표_데이터 = pd.read_html(현금흐름표.get_attribute('outerHTML'))[0]


    # 재무상태표 가공
    재무상태표_가공 = 재무상태표_데이터.drop('주석', axis=1)
    재무상태표_가공 = 재무상태표_가공.replace('-', '0')
    for i in range(1, 5):
        재무상태표_가공.iloc[:, i] = 재무상태표_가공.iloc[:, i].astype(str).str.replace('(', '-').str.replace(')', '').str.replace(',', '')
    재무상태표_가공.iloc[:, 1:] = 재무상태표_가공.iloc[:, 1:].astype(float)
    재무상태표_가공.columns = ['과 목', '당기', '당기2', '전기', '전기2']
    재무상태표_가공.insert(0, '분류', '재무상태표')
    재무상태표_가공.insert(0, '기준년도', 기준년도)

    # 손익계산서 가공
    손익계산서_가공 = 손익계산서_데이터.drop('주석', axis=1)
    손익계산서_가공 = 손익계산서_가공.replace('-', '0')
    for i in range(1, 5):
        손익계산서_가공.iloc[:, i] = 손익계산서_가공.iloc[:, i].astype(str).str.replace('(', '-').str.replace(')', '').str.replace(',', '')
    손익계산서_가공.iloc[:, 1:] = 손익계산서_가공.iloc[:, 1:].astype(float)
    손익계산서_가공.columns = ['과 목', '당기', '당기2', '전기', '전기2']
    손익계산서_가공.insert(0, '분류', '손익계산서')
    손익계산서_가공.insert(0, '기준년도', 기준년도)

    # 자본변동표 가공
    #자본변동표_가공 = 자본변동표_데이터.copy()
    #for i in range(1, 8):
    #    자본변동표_가공.iloc[:, i] = 자본변동표_가공.iloc[:, i].astype(str).str.replace('-', '0').str.replace('(', '-').str.replace(')', '').str.replace(',', '')
    #자본변동표_가공.iloc[:, 1:] = 자본변동표_가공.iloc[:, 1:].astype(float)
    #자본변동표_가공.columns = 자본변동표_가공.columns.levels[1][자본변동표_가공.columns.codes[1]]

    # 현금흐름표 가공
    현금흐름표_가공 = 현금흐름표_데이터.copy()
    현금흐름표_가공 = 현금흐름표_가공.replace('-', '0')
    for i in range(1, 5):
        현금흐름표_가공.iloc[:, i] = 현금흐름표_가공.iloc[:, i].astype(str).str.replace('(', '-').str.replace(')', '').str.replace(',', '')
    현금흐름표_가공.iloc[:, 1:] = 현금흐름표_가공.iloc[:, 1:].astype(float)
    현금흐름표_가공.columns = ['과 목', '당기', '당기2', '전기', '전기2']
    현금흐름표_가공.insert(0, '분류', '현금흐름표')
    현금흐름표_가공.insert(0, '기준년도', 기준년도)
    
    # 적재
    재무상태표_집계.append(재무상태표_가공)
    손익계산서_집계.append(손익계산서_가공)
    현금흐름표_집계.append(현금흐름표_가공)
    
재무상태표_집계 = pd.concat(재무상태표_집계)
손익계산서_집계 = pd.concat(손익계산서_집계)
현금흐름표_집계 = pd.concat(현금흐름표_집계)
재무제표_집계 = pd.concat([재무상태표_집계, 손익계산서_집계, 현금흐름표_집계])

In [211]:
writer = pd.ExcelWriter('dart.xlsx', 'xlsxwriter')
재무제표_집계.to_excel(writer, index=False)
writer.save()
writer.close()